<a href="https://colab.research.google.com/github/wagnercastro1000-star/Algoritmo-de-ML---Titanic/blob/Com-LinearSVC/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este desafio  de Machine Learning (ML) está disponível no site do Kaggle ("https://www.kaggle.com/") e consiste em tentar criar um algoritmo que, após ser treinado com os os dados de treino disponibilizados pelo próprio site ("train_data"), preveja quais dos passageiros do famoso navio Titanic sobreviveram e quais não, analisando os dados de teste ("test_data"), podendo-se comparar depois o índice de acerto, já que é sabido quem sobreviveu e quem não ao naufrágio.  

Começo fazendo as importações das bibliotecas e dos dados, verificando depois se tudo funcionou bem.

In [42]:
import numpy as np
import pandas as pd

Eis os dados de treinamento abaixo, já separados pelo Kaggle - após baixá-los eu fiz o upload no Google Colab:

In [43]:
train_data = pd.read_csv("/content/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Agora, eis os dados de teste, também já disponibilizados pelo Kaggle:

In [44]:
test_data = pd.read_csv("/content/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


O próprio Kaggle também disponibiliza a informação abaixo, de que mulheres sobreviveram mais do que os homens, bastante supor que 100% das mulheres sobreviveram para ter um índice de acerto de 74%.

A taxa de sobrevivência das mulheres foi de 74%, conforme vemos abaixo. Isto significa que temos que achar um algoritmo que supere a simples suposição que as mulheres sobreviveram e os homens não.

In [45]:
female = train_data.loc[train_data.Sex == 'female']['Survived']
female_survival_rate = sum(female) / len(female)

print("Female survival rate: ", female_survival_rate)

Female survival rate:  0.7420382165605095


Agora que já nos familiarizamos com o problema, vamos começar a inspecionar os dados. Podemos fazer isto usando alguns métodos como o (já usado acima) .head(), .tail(), .sample() e outros. Vou usar o .shape e o .describe().


In [46]:
train_data.shape

(891, 12)

In [47]:
test_data.shape

(418, 11)

Vemos que os dados de treino têm 891 linhas e 12 colunas e os dados de teste têm 418 linhas e 11 colunas e isto porque nos dados de teste não há uma coluna indicando se o passageiro sobreviveu ou não - como o Kaggle é uma competição, estes dados são secretos.
Vamos agora inspecionar os dados através do método .describe().

In [48]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Algumas colunas são auto-explicativas, porém vale uma observação sobre aquelas que não são:
Pclass = se o passageiro viajou de 1ª, 2ª ou 3ª classes;
SibSp = se o passageiro tinha irmãos ou cônjuges a bordo do Titanic;
Parch = se o passageiro tinha pais ou filhos a bordo;
Fare = quanto foi pago pela passagem.




In [49]:
test_data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


Outra observação é que apenas dados numéricos apareceram - nome, cabine, local de embarque e sexo não apareceram. Podemos ver no entanto que no train_data temos 891 passageiros e, destes, faltam 177 dados sobre a idade (no test_data faltam 86 dados sobre a idade, confira), o que terá de ser tratado antes de os dados serem utilizados. Vamos tentar ver agora as colunas não numéricas fazendo uma especificação no método .describe().

In [50]:
train_data.describe(include='object')

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Dooley, Mr. Patrick",male,347082,G6,S
freq,1,577,7,4,644


In [51]:
test_data.describe(include='object')

,Name,Sex,Ticket,Cabin,Embarked
count,418,418,418,91,418
unique,418,2,363,76,3
top,"Peter, Master. Michael J",male,PC 17608,B57 B59 B63 B66,S
freq,1,266,5,3,270


Com relação aos demais dados, notamos que faltam muitos dados sobre o número da cabine em que estavam os passageiros, de modo que este é um dado que dificilmente será útil para nós. E nos dados de treino faltam apenas 2 registros com relação ao porto de embarque.

Por enquanto, esta inspeção deu-me algumas ideias de como tentar explorar os dados, e por isso não vou utilizar o Pandas Profiling. Caso não tenhamos sucesso, podemos usá-lo posteriormente.

Resumindo, os dados possuem alguns problemas que precisam ser tratados:
1. Faltam 177 dados sobre a idade do passageiro no train_data e 86 no test_data;
2. Faltam muitos dados sobre o número da cabine em que estavam os passageiros;
3. Faltam apenas 2 registros com relação ao porto de embarque (isto poderá ser ignorado);
4. Além disso, dados em string como o Sexo do passageiro deverão ser convertidos em dados numéricos, dependendo do algoritmo que será utilizado.

Vamos passar agora para a fase de tratamento dos dados. Vamos confirmar primeiro se há realmente dados faltantes na coluna Idade e se são 177 e 86 mesmo, como havíamos observado:

In [52]:
train_data["Age"].isnull().sum()

np.int64(177)

In [53]:
test_data["Age"].isnull().sum()

np.int64(86)

Confirmada a nossa verificação, vamos tratar estes dados. Poderíamos excluir estes dados, substituí-los por zero, usar a média ou a mediana no lugar dos dados faltantes. A minha opção será usar a mediana:

In [54]:
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())

Vejamos se deu certo:

In [55]:
train_data["Age"].isnull().sum()

np.int64(0)

Agora vamos fazer o mesmo com os dados do test_data e depois veremos se deu certo.

In [56]:
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())

In [57]:
test_data["Age"].isnull().sum()

np.int64(0)

Agora, vamos transformar os dados em string da coluna "SEX" em dados numéricos.

In [58]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_data["Sex"] = le.fit_transform(train_data["Sex"])
test_data["Sex"] = le.transform(test_data["Sex"])

In [59]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [60]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,S


# Neste caso, homens terão valor "1" e mulheres valor "0" - uma dica do Gemini 🙂.

Vamos passar agora para a próxima fase, que será a escolha do algoritmo de Machine Learning que usaremos. Para isso, temos que ter também algumas hipóteses que iremos testar:
1. Mulheres sobreviveram mais do que os homens - isto já é sabido;
2. Vou supor também que crianças sobreviveram mais ("são salvos primeiro mulheres e crianças"), de modo que a coluna "Age" também será importante;
3. Em terceiro lugar, vou supor que passageiros de classe social mais elevado sobreviveram mais, de modo que as colunas "Fare" e "Pclass" também serão consideradas como importantes.
4. Vou supor que as demais colunas não contém dados importantes para prever a sobrevivência ou não do passageiro.  

Para escolher o algoritmo, consultei o "Sklearn cheat sheet" ("https://scikit-learn.org/stable/machine_learning_map.html"), uma árvore de decisão que facilita na escolha do algoritmo de ML a ser usado. Os algoritmos LinearSVC ou SGDClassifier foram os mais indicados para o nosso problema. Eu havia optado anteriormente pelo SGDClassifier, com um índice de acerto de 76,8%. Agora tentarei usar o Linear SVC.  

Vamos entãos fazer as importações e preparar os dados, conforme a documentação do sklearn consultada no link acima:

In [61]:
from sklearn import svm

In [62]:
X = train_data[["Sex", "Age", "Fare", "Pclass"]]
y = train_data["Survived"]

In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

modelo = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", svm.SVC())
])

In [64]:
test_data["Fare"].isnull().sum()

np.int64(1)

Há um dado faltante na coluna "Fare" do test_data, o que resultou em um problema na hora de treinar o modelo, vamos corrigir colocando a média no lugar do dado faltante.

In [65]:
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].mean())

Agora vamos treinar o modelo:

In [66]:
modelo.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()), ('clf', SVC())])

Agora usamos o modelo treinado para fazer a previsão (deseje-me sorte!).

In [67]:
y_pred = modelo.predict(test_data[["Sex", "Age", "Fare", "Pclass"]])

Criando o arquivo para enviar ao Kaggle:

In [68]:
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": y_pred
})


Transformando em csv e verificando com o método .head() se está tudo ok. Após isto faremos o download do arquivo.

In [69]:
submission.to_csv("submission.csv", index=False)

In [70]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [71]:
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Nota do Kaggle para o LinearSVC: 0.77272, 77,2%; uma pequena melhora em relação à nota anterior com o SGDClassifier: 0.76794 ≈ 76,8% de acerto.